# churn prediction

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, precision_score, recall_score, precision_recall_curve
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [209]:
data=pd.read_csv(r"C:\Users\ADMIN\Downloads\churn_prediction (1).csv")

In [210]:
data.head(8)

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,1,3135,66,Male,0.0,self_employed,187.0,2,755,224.0,...,1458.71,1458.71,1449.07,0.20,0.20,0.20,0.20,1458.71,1458.71,0
1,2,310,35,Male,0.0,self_employed,NaN,2,3214,60.0,...,8704.66,7799.26,12419.41,0.56,0.56,5486.27,100.56,6496.78,8787.61,0
2,4,2356,31,Male,0.0,salaried,146.0,2,41,NaN,...,5815.29,4910.17,2815.94,0.61,0.61,6046.73,259.23,5006.28,5070.14,0
3,5,478,90,NaN,NaN,self_employed,1020.0,2,582,147.0,...,2291.91,2084.54,1006.54,0.47,0.47,0.47,2143.33,2291.91,1669.79,1
4,6,2531,42,Male,2.0,self_employed,1494.0,3,388,58.0,...,1401.72,1643.31,1871.12,0.33,714.61,588.62,1538.06,1157.15,1677.16,1
5,7,263,42,Female,0.0,self_employed,1096.0,2,1666,60.0,...,16059.34,15211.29,13798.82,0.36,0.36,857.50,286.07,15719.44,15349.75,0
6,8,5922,72,Male,0.0,retired,1020.0,1,1,98.0,...,7714.19,7859.74,11232.37,0.64,0.64,1299.64,439.26,7076.06,7755.98,0
7,9,1145,46,Male,0.0,self_employed,623.0,2,317,172.0,...,8519.53,6511.82,16314.17,0.27,0.27,443.13,5688.44,8563.84,5317.04,0


In [211]:
data.tail(9)

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
28373,30293,2384,73,Male,0.0,retired,1381.0,3,790,3.0,...,1926.57,1491.87,1973.16,2514.47,433.49,1039.76,180.37,1410.80,1800.70,0
28374,30294,658,55,Male,0.0,salaried,1076.0,3,463,306.0,...,7713.88,7713.88,7670.06,0.33,0.33,0.33,0.33,7713.88,7713.88,1
28375,30295,2041,42,Male,0.0,self_employed,146.0,2,286,57.0,...,6777.41,8082.48,3515.03,0.51,90.10,1103.20,1183.04,7956.03,7431.36,0
28376,30296,2155,85,Male,0.0,retired,1589.0,2,389,NaN,...,1741.50,1741.50,1730.46,0.04,0.04,0.04,0.04,1741.50,1741.50,0
28377,30297,1845,10,Female,0.0,student,1020.0,2,1207,70.0,...,1076.43,2282.19,2787.70,0.30,0.30,0.30,0.30,1076.43,1076.43,0
28378,30298,4919,34,Female,0.0,self_employed,1046.0,2,223,14.0,...,4069.21,3668.83,3865.55,1.71,2.29,901.00,1014.07,3738.54,3690.32,0
28379,30299,297,47,Male,0.0,salaried,1096.0,2,588,0.0,...,61017.55,53444.81,21925.81,4666.84,3883.06,168.23,71.80,61078.50,57564.24,1
28380,30300,2585,50,Male,3.0,self_employed,1219.0,3,274,NaN,...,1625.55,1683.20,1857.42,0.20,0.20,0.20,0.20,1625.55,1625.55,0
28381,30301,2349,18,Male,0.0,student,1232.0,2,474,59.0,...,2821.34,3213.44,4447.45,0.11,7.44,714.40,1094.09,2402.62,3260.58,1


In [212]:
data.describe()

,customer_id,vintage,age,dependents,city,customer_nw_category,branch_code,days_since_last_transaction,current_balance,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
count,28382.000000,28382.000000,28382.000000,25919.000000,27579.000000,28382.000000,28382.000000,25159.000000,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,2.838200e+04,28382.000000
mean,15143.508667,2364.336446,48.208336,0.347236,796.109576,2.225530,925.975019,69.997814,7.380552e+03,7.495771e+03,7.496780e+03,7.124209e+03,3.433252e+03,3.261694e+03,3.658745e+03,3.339761e+03,7.451133e+03,7.495177e+03,0.185329
std,8746.454456,1610.124506,17.807163,0.997661,432.872102,0.660443,937.799129,86.341098,4.259871e+04,4.252935e+04,4.172622e+04,4.457581e+04,7.707145e+04,2.968889e+04,5.198542e+04,2.430111e+04,4.203394e+04,4.243198e+04,0.388571
min,1.000000,180.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,-5.503960e+03,-3.149570e+03,1.428690e+03,-1.650610e+04,1.000000e-02,1.000000e-02,1.000000e-02,1.000000e-02,-3.374180e+03,-5.171920e+03,0.000000
25%,7557.250000,1121.000000,36.000000,0.000000,409.000000,2.000000,176.000000,11.000000,1.784470e+03,1.906000e+03,2.180945e+03,1.832507e+03,3.100000e-01,3.300000e-01,4.100000e-01,4.100000e-01,1.996765e+03,2.074407e+03,0.000000
50%,15150.500000,2018.000000,46.000000,0.000000,834.000000,2.000000,572.000000,30.000000,3.281255e+03,3.379915e+03,3.542865e+03,3.359600e+03,6.100000e-01,6.300000e-01,9.193000e+01,1.099600e+02,3.447995e+03,3.465235e+03,0.000000
75%,22706.750000,3176.000000,60.000000,0.000000,1096.000000,3.000000,1440.000000,95.000000,6.635820e+03,6.656535e+03,6.666887e+03,6.517960e+03,7.072725e+02,7.492350e+02,1.360435e+03,1.357553e+03,6.667958e+03,6.654693e+03,0.000000
max,30301.000000,12899.000000,90.000000,52.000000,1649.000000,3.000000,4782.000000,365.000000,5.905904e+06,5.740439e+06,5.700290e+06,5.010170e+06,1.226985e+07,2.361808e+06,7.637857e+06,1.414168e+06,5.778185e+06,5.720144e+06,1.000000


In [213]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28382 entries, 0 to 28381
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   customer_id                     28382 non-null  int64  
 1   vintage                         28382 non-null  int64  
 2   age                             28382 non-null  int64  
 3   gender                          27857 non-null  object 
 4   dependents                      25919 non-null  float64
 5   occupation                      28302 non-null  object 
 6   city                            27579 non-null  float64
 7   customer_nw_category            28382 non-null  int64  
 8   branch_code                     28382 non-null  int64  
 9   days_since_last_transaction     25159 non-null  float64
 10  current_balance                 28382 non-null  float64
 11  previous_month_end_balance      28382 non-null  float64
 12  average_monthly_balance_prevQ   

In [214]:
data.shape

(28382, 21)

In [215]:
data.isnull().sum()

customer_id                          0
vintage                              0
age                                  0
gender                             525
dependents                        2463
occupation                          80
city                               803
customer_nw_category                 0
branch_code                          0
days_since_last_transaction       3223
current_balance                      0
previous_month_end_balance           0
average_monthly_balance_prevQ        0
average_monthly_balance_prevQ2       0
current_month_credit                 0
previous_month_credit                0
current_month_debit                  0
previous_month_debit                 0
current_month_balance                0
previous_month_balance               0
churn                                0
dtype: int64

In [216]:
data.gender.value_counts()

Male      16548
Female    11309
Name: gender, dtype: int64

In [217]:
dict_gender= {'Male': 1, 'Female':0}
data.replace({'gender': dict_gender}, inplace = True)

data['gender'] = data['gender'].fillna(-1)

In [218]:
data.gender

0        1.0
1        1.0
2        1.0
3       -1.0
4        1.0
        ... 
28377    0.0
28378    0.0
28379    1.0
28380    1.0
28381    1.0
Name: gender, Length: 28382, dtype: float64

In [219]:
data.dependents.value_counts()

0.0     21435
2.0      2150
1.0      1395
3.0       701
4.0       179
5.0        41
6.0         8
7.0         3
9.0         1
52.0        1
36.0        1
50.0        1
8.0         1
25.0        1
32.0        1
Name: dependents, dtype: int64

In [220]:
data['dependents'] = data['dependents'].fillna(0)

In [221]:
data.dependents

0        0.0
1        0.0
2        0.0
3        0.0
4        2.0
        ... 
28377    0.0
28378    0.0
28379    0.0
28380    3.0
28381    0.0
Name: dependents, Length: 28382, dtype: float64

In [222]:
data.occupation.value_counts()

self_employed    17476
salaried          6704
student           2058
retired           2024
company             40
Name: occupation, dtype: int64

In [223]:
data['occupation'] = data['occupation'].fillna('self_employed')

In [224]:
data.occupation

0        self_employed
1        self_employed
2             salaried
3        self_employed
4        self_employed
             ...      
28377          student
28378    self_employed
28379         salaried
28380    self_employed
28381          student
Name: occupation, Length: 28382, dtype: object

In [225]:
data.city.value_counts()

1020.0    3479
1096.0    2016
409.0     1334
146.0     1291
834.0     1138
          ... 
748.0        1
1099.0       1
108.0        1
655.0        1
723.0        1
Name: city, Length: 1604, dtype: int64

In [226]:
data['city'] = data['city'].fillna(1020)

In [227]:
data.city

0         187.0
1        1020.0
2         146.0
3        1020.0
4        1494.0
          ...  
28377    1020.0
28378    1046.0
28379    1096.0
28380    1219.0
28381    1232.0
Name: city, Length: 28382, dtype: float64

In [228]:
data.days_since_last_transaction.value_counts()

0.0      1672
3.0       831
14.0      654
4.0       632
6.0       596
         ... 
309.0       1
225.0       1
183.0       1
92.0        1
334.0       1
Name: days_since_last_transaction, Length: 360, dtype: int64

In [229]:
data['days_since_last_transaction'] = data['days_since_last_transaction'].fillna(999)

In [230]:
data.days_since_last_transaction

0        224.0
1         60.0
2        999.0
3        147.0
4         58.0
         ...  
28377     70.0
28378     14.0
28379      0.0
28380    999.0
28381     59.0
Name: days_since_last_transaction, Length: 28382, dtype: float64

In [231]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [232]:
for obj in data:
    data[obj]=le.fit_transform(data[obj].astype(str))
data.head(8)    

,customer_id,vintage,age,gender,dependents,occupation,city,customer_nw_category,branch_code,days_since_last_transaction,...,previous_month_end_balance,average_monthly_balance_prevQ,average_monthly_balance_prevQ2,current_month_credit,previous_month_credit,current_month_debit,previous_month_debit,current_month_balance,previous_month_balance,churn
0,0,2165,62,2,0,3,724,1,2955,138,...,3484,2211,3700,16,15,13,13,3450,3210,0
1,10395,2129,28,2,0,3,23,1,2032,317,...,26737,26050,2077,47,49,10199,122,23908,26877,0
2,22136,1349,24,2,0,2,494,1,2509,360,...,22624,21563,12824,52,53,10768,5892,21005,21569,0
3,23188,3550,89,0,0,3,23,1,2793,53,...,10305,8935,89,38,40,33,4801,10180,5405,1
4,24226,1531,36,2,2,3,532,2,2404,314,...,2914,4623,7304,27,9159,10614,2832,1233,5482,1
5,25268,1635,36,1,0,3,103,1,614,317,...,4995,3117,3128,29,29,12886,6487,4609,4068,0
6,26315,4448,69,2,0,1,23,0,0,358,...,25614,26123,1118,55,56,1550,9186,24806,25887,0
7,27353,142,40,2,0,3,1196,1,2002,81,...,26515,24527,5432,22,22,9092,10597,26643,22094,0


In [233]:
num_cols = ['customer_nw_category', 'current_balance',
            'previous_month_end_balance', 'average_monthly_balance_prevQ2', 'average_monthly_balance_prevQ',
            'current_month_credit','previous_month_credit', 'current_month_debit',
            'previous_month_debit','current_month_balance', 'previous_month_balance']
for i in num_cols:
    data[i] = np.log(data[i] + 17000)

std = StandardScaler()
scaled = std.fit_transform(data[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

In [234]:
data_data_og = data.copy()
data = data.drop(columns = num_cols,axis = 1)
data = data.merge(scaled,left_index=True,right_index=True,how = "left")

In [235]:
y_all = data['churn']
data = data.drop(['churn','customer_id'],axis = 1)

In [236]:
baseline_cols = ['current_month_debit', 'previous_month_debit','current_balance','previous_month_end_balance','vintage'
                 ,'occupation']

In [237]:
data_baseline = data[baseline_cols]

In [238]:
xtrain, xtest, ytrain, ytest = train_test_split(data_baseline,y_all,test_size=1/3, random_state=11, stratify = y_all)

In [239]:
from sklearn.linear_model import LogisticRegression

In [240]:
r=LogisticRegression()

In [241]:
reg=r.fit(xtrain,ytrain)

In [242]:
y_pred=r.predict(xtest)

In [243]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [244]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,y_pred)

0.8147130324490012